In [ ]:
!wget https://github.com/rycolab/pareto-probing/archive/master.zip
!unzip master.zip
!pip install transformers==3.5.1
!pip install git+https://github.com/facebookresearch/fastText
!pip install seaborn ipdb conllu

In [ ]:
!make get_ud

In [ ]:
!make process LANGUAGE=english REPRESENTATION=onehot
!make process LANGUAGE=english REPRESENTATION=bert

In [ ]:
!python -u src/h02_learn/random_search.py --language english --data-path ./data/processed --representation onehot --checkpoint-path ./checkpoints --task pos_tag --model linear

2020-12-04 14:23:01.263627: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(alpha=0.3, batch_size=2048, checkpoint_path='./checkpoints', data_path='./data/processed', dropout=0.2, embedding_size=300, eval_batches=100, hidden_size=512, language='english', max_rank=10, model='linear', nlayers=3, representation='onehot', save_path='./checkpoints/pos_tag/english/linear/onehot/nl_3-es_300-hs_512-d_0.2000-a_0.3000', seed=20, shuffle_labels=False, task='pos_tag', wait_epochs=10, wait_iterations=1000)
100% 50/50 [00:00<00:00, 443372.52it/s]


In [ ]:
!python -u src/h02_learn/random_search.py --language english --data-path ./data/processed --representation bert --checkpoint-path ./checkpoints_new --task pos_tag --model linear

In [ ]:
import pathlib
import os
import pickle

def write_data(filename, data):
    with open(filename, "wb") as f:
        pickle.dump(data, f)


def read_data(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data

def mkdir(folder):
    pathlib.Path(folder).mkdir(parents=True, exist_ok=True)

language = 'english'
output_path = './data/processed'
output_path = os.path.join(output_path, language)
mkdir(output_path)
output_file_base = os.path.join(output_path, '%s--%s.pickle.bz2')
for mode in ['train', 'dev', 'test']:
		input_file = output_file_base % (mode, 'bert')
		bert_embeddings = read_data(input_file)
		negative_bert = [-i for i in bert_embeddings]
		write_data(output_file_base % (mode, 'negative_bert'), negative_bert)

In [ ]:
!python -u src/h02_learn/random_search.py --language english --data-path ./data/processed --representation negative_bert --checkpoint-path ./checkpoints_new --task pos_tag --model linear

2020-12-04 06:43:15.424193: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(alpha=0.3, batch_size=2048, checkpoint_path='./checkpoints_new', data_path='./data/processed', dropout=0.2, embedding_size=300, eval_batches=100, hidden_size=512, language='english', max_rank=10, model='linear', nlayers=3, representation='negative_bert', save_path='./checkpoints_new/pos_tag/english/linear/negative_bert/nl_3-es_300-hs_512-d_0.2000-a_0.3000', seed=20, shuffle_labels=False, task='pos_tag', wait_epochs=10, wait_iterations=1000)
Traceback (most recent call last):
  File "src/h02_learn/random_search.py", line 197, in <module>
    main()
  File "src/h02_learn/random_search.py", line 174, in main
    search = get_hyperparameters_search(n_runs, args.representation, n_classes)
  File "src/h02_learn/random_search.py", line 64, in get_hyperparameters_search
    raise ValueError('Invalid representation %s' % representation)
Valu

In [ ]:
!python -u src/h02_learn/random_search.py --language english --data-path ./data/processed --representation d1_bert --checkpoint-path ./checkpoints_new --task pos_tag --model linear

2020-12-04 06:43:46.971289: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(alpha=0.3, batch_size=2048, checkpoint_path='./checkpoints_new', data_path='./data/processed', dropout=0.2, embedding_size=300, eval_batches=100, hidden_size=512, language='english', max_rank=10, model='linear', nlayers=3, representation='d1_bert', save_path='./checkpoints_new/pos_tag/english/linear/d1_bert/nl_3-es_300-hs_512-d_0.2000-a_0.3000', seed=20, shuffle_labels=False, task='pos_tag', wait_epochs=10, wait_iterations=1000)
Traceback (most recent call last):
  File "src/h02_learn/random_search.py", line 197, in <module>
    main()
  File "src/h02_learn/random_search.py", line 174, in main
    search = get_hyperparameters_search(n_runs, args.representation, n_classes)
  File "src/h02_learn/random_search.py", line 64, in get_hyperparameters_search
    raise ValueError('Invalid representation %s' % representation)
ValueError: Inva

In [ ]:
!python -u src/h03_analysis/compile_results.py --checkpoint-path checkpoints_new/ --task pos_tag

In [ ]:
import math
import numpy as np
def generate_remove(number_to_remove): 
  np.random.seed(768)
  idx = np.random.choice(768, number_to_remove, replace=False)
  for mode in ['train', 'dev', 'test']:
      input_file = output_file_base % (mode, 'bert')
      bert_embeddings = read_data(input_file)
      del_bert = []
      for embeddings in bert_embeddings:
        temp = embeddings.copy()
        temp[:,idx] = 0
        del_bert.append(temp)
      write_data(output_file_base % (mode, 'd%d_bert'%number_to_remove), del_bert)
  return
for i in range(10):
  temp = math.ceil(768*i/10+1)
  generate_remove(temp)
  print(temp)

In [ ]:
def generate_inv(output_file_base): 
  np.random.seed(768)
  matrix = 0
  while True:
    matrix = np.random.rand(768,768).astype(np.float32)
    if np.linalg.matrix_rank(matrix) == 768:
      # matrix = matrix/np.linalg.norm(matrix)
      break
  for mode in ['train', 'dev', 'test']:
    input_file = output_file_base % (mode, 'bert')
    bert_embeddings = read_data(input_file)
    inv_bert = []
    for embeddings in bert_embeddings:
      temp = embeddings.copy()
      temp = temp.dot(matrix)
      inv_bert.append(temp)
    write_data(output_file_base % (mode, 'inv_bert'), inv_bert)
  return
generate_inv(output_file_base)

In [ ]:
embeddings

In [ ]:
!python -u src/h02_learn/random_search.py --language english --data-path ./data/processed --representation inv_bert --checkpoint-path ./checkpoints_new --task pos_tag --model linear

In [ ]:
from sklearn import datasets
import pathlib
import os
import numpy as np
import torch

X, y = datasets.make_classification(n_samples = 1000, n_features = 10, n_informative = 2, n_classes = 2)
# print(X[10])

train_cls_dict = {"X":X[:700], "y":y[:700]}
val_cls_dict = {"X":X[700:850], "y":y[700:850]}
test_cls_dict = {"X":X[850:], "y":y[850:]}
torch.save(train_cls_dict, "./data/cls/train-cls.pth")
torch.save(val_cls_dict, "./data/cls/dev-cls.pth")
torch.save(test_cls_dict, "./data/cls/test-cls.pth")

In [ ]:
!python -u src/h02_learn/random_search.py --language english --data-path ./data/cls --representation vector --task classify --checkpoint-path ./checkpoints/cls/ --model linear

2020-12-04 17:46:21.959118: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(alpha=0.3, batch_size=2048, checkpoint_path='./checkpoints/cls/', data_path='./data/cls', dropout=0.2, embedding_size=300, eval_batches=100, hidden_size=512, language='english', max_rank=10, model='linear', nlayers=3, representation='vector', save_path='./checkpoints/cls/classify/english/linear/vector/nl_3-es_300-hs_512-d_0.2000-a_0.3000', seed=20, shuffle_labels=False, task='classify', wait_epochs=10, wait_iterations=1000)
['vector', '--hidden-size', '776', '--nlayers', '0', '--dropout', '0.15', '--embedding-size', '10', '--alpha', '0.0', '--max-rank', '1']
['vector', '--hidden-size', '433', '--nlayers', '2', '--dropout', '0.27', '--embedding-size', '10', '--alpha', '0.0009765625', '--max-rank', '1']
['vector', '--hidden-size', '310', '--nlayers', '5', '--dropout', '0.5', '--embedding-size', '10', '--alpha', '0.0011782270808598852'

In [1]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
cd ..

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!git clone https://github.com/cornbluecloud/pareto-probing.git

Cloning into 'pareto-probing'...
remote: Enumerating objects: 1068, done.
remote: Counting objects: 100% (1068/1068), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 10798 (delta 235), reused 1065 (delta 232), pack-reused 9730
Receiving objects: 100% (10798/10798), 236.47 MiB | 12.89 MiB/s, done.
Resolving deltas: 100% (604/604), done.
Checking out files: 100% (5741/5741), done.


In [ ]:
cd pareto-probing/

/content/drive/My Drive/Colab Notebooks/pareto-probing


In [ ]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   multiple_bijections/R00_d0_100_bert/all_results.tsv
	modified:   multiple_bijections/R00_d80_100_bert/all_results.tsv

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	pareto-probing-newdata/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add pareto-probing-newdata/

In [ ]:
cp 